In [1]:
import math
import pickle
import joblib
import numpy as np
import pandas as pd
from flask import Flask, jsonify, request
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [2]:
# https://www.tutorialspoint.com/flask
import flask
app = Flask(__name__)

In [ ]:
#Reference for Query Point Formatting

#1. Query Point Format - List
'''query_pt_features = ['timestamp','building_id', 'meter', 'primary_use', 'square_feet',
        'floor_count', 'air_temperature', 'cloud_coverage', 'dew_temperature',
        'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
        'wind_speed']

#2. Building Usage- Category to choose from
primary_use_list = ['Retail','Parking','Office','Education','Lodging/residential', 'Entertainment/public assembly',
                    'Public services', 'Warehouse/storage','Food sales and service', 'Religious worship','Healthcare', 
                    'Utility','Other', 'Technology/science','Manufacturing/industrial','Services']'''

"query_pt_features = ['timestamp','building_id', 'meter', 'primary_use', 'square_feet',\n        'floor_count', 'air_temperature', 'cloud_coverage', 'dew_temperature',\n        'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',\n        'wind_speed']\n\n#2. Building Usage- Category to choose from\nprimary_use_list = ['Retail','Parking','Office','Education','Lodging/residential', 'Entertainment/public assembly',\n                    'Public services', 'Warehouse/storage','Food sales and service', 'Religious worship','Healthcare', \n                    'Utility','Other', 'Technology/science','Manufacturing/industrial','Services']"

In [ ]:
def final(query_pt:"Input Query Point in list format",
          LabelEncoder_dict:"Pre-trained Label Encoders for categorical features", 
          model:"Pretrained Regression Model", 
          y:"Ground Truth"=None ) -> "It returns the Energy Prediction":

  '''
      1.This function accepts the query points in a list format, performs all the required 
      data cleaning, preprocessing and featurization steps to make it compatible with 
      the model and predicts the energy usage. 
      2. This also compute error if the ground thuth i.e. y is provided.

      3. Query Point format is as below:

      ['timestamp','building_id', 'meter', 'primary_use', 'square_feet', 'floor_count', 
       'air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 
       'sea_level_pressure', 'wind_direction', 'wind_speed']
  '''

  # 1. Preparing DataFrame from query point.
  query_pt_features = ['timestamp','building_id', 'meter', 'primary_use', 'square_feet',
                       'floor_count', 'air_temperature', 'cloud_coverage', 'dew_temperature',
                       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
                       'wind_speed']  
  query_df = pd.DataFrame(query_pt).T
  query_df.columns = query_pt_features

  # 2. Replacing floor_count feature with floor_info
  query_df.floor_count.fillna(0, inplace=True)
  query_df['floor_count'] = query_df.floor_count.apply(lambda x: 1 if x!=0 else 0)
  query_df.rename(columns={"floor_count":"floor_info"}, inplace=True)

  # 3. Converting Timestamp dataformat to timestamp for feature extractions
  query_df['timestamp'] = pd.to_datetime(query_df.timestamp)

  query_df['year'] = query_df['timestamp'].dt.year
  query_df['month'] = query_df['timestamp'].dt.month
  query_df['day_of_Week'] = query_df['timestamp'].dt.dayofweek
  query_df['day_name'] = query_df['timestamp'].dt.day_name()
  query_df['hour'] = query_df['timestamp'].dt.hour

  # 4. Ensure Rest of the futures are Numeric 
  query_df = query_df._convert(numeric=True)

  # 5. Adding Season Feature 
  seasons = {(1, 12, 2): 1, (3, 4, 5): 2, (6, 7, 8): 3, (9, 10, 11): 4}
  def season(month):
      for month_table in seasons.keys():
          if month in list(month_table):
              return seasons[month_table] 
  query_df['season'] = query_df.month.apply(season)

  # 6. Adding New Feature - Relative Humidity:
  query_df['RH'] = np.round((0.608 * np.exp(float((17.27*query_df.dew_temperature)/(query_df.dew_temperature+237.3)), dtype=float) /\
                            0.608 * np.exp(float((17.27*query_df.air_temperature)/(query_df.air_temperature+237.3)), dtype=float))*100,2)

  # 7. Adding New Feature - Cloud Base
  query_df['Cloud_base'] = (((((query_df.air_temperature*1.8)+32) - ((query_df.dew_temperature*1.8)+32))/4.4)*1000)*0.3048

  # 8. Adding New Feature - Windchill Index (WCI):
  query_df['WCI'] = np.round(float((13.12 + 0.6215 * query_df.air_temperature - 11.37*((query_df.wind_speed*2.235)**0.16) \
                    + 0.3965* query_df.air_temperature * ((query_df.wind_speed*2.235)**0.16))),2)

  # 9. Adding New Feature - DAY/NIGHT Feature:
  day_hours = [i for i in range(6,19)]
  query_df['D/N'] = query_df.hour.apply(lambda x: 'Day' if x in day_hours else 'Night')

  # 10. Discarding Unwanted Features
  query_df = query_df.drop(['timestamp', 'dew_temperature', 'year'],axis=1)

  # 11. Encoding Categorical Data
  #with open('/content/drive/MyDrive/Case Study 01/Data/Label Encoder/LabelEncoder.bin', 'rb') as f:
      #LabelEncoder_dict = pickle.load(f)
  DN_encoder, day_name_encoder, primary_use_encoder = LabelEncoder_dict['DN_encoder'], LabelEncoder_dict['day_name_encoder'], LabelEncoder_dict['primary_use_encoder']

  categorical_columns = query_df.select_dtypes(include=['category', 'object']).columns.tolist()
  encoders = [primary_use_encoder, day_name_encoder, DN_encoder]

  for i in range(len(categorical_columns)):
    feature = categorical_columns[i]
    encoder = encoders[i]
    query_df[feature]= encoder.transform(np.array(query_df[feature]).reshape(1,))

  # 12. Predictions
  query_pred = np.round(np.expm1(model.predict(query_df))[0],2)

  # 13. Computing error if actual value of y is given
  if y:
    RMSE = RMSE = np.round((np.sqrt(np.square(y - query_pred))/1),2)
  else:
    RMSE = np.nan

  return query_pred, RMSE

In [ ]:
@app.route('/')
def hello_world():
    return 'Hello World!!'


@app.route('/index')
def index():
    return flask.render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
    #clf = joblib.load('model.pkl')
    LGBM_reg_best = joblib.load('LGBM_reg_Rev02.sav')

    #count_vect = joblib.load('count_vect.pkl')
    LabelEncoder_dict = joblib.load('LabelEncoder.bin')

    to_predict_list = request.form.to_dict()
    query_pred, RMSE = final(query_pt, LabelEncoder_dict, LGBM_reg_best, y=None )

    return jsonify({'Predicted Energy Consumption': query_pred, 'RMSE':RMSE})


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8060)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8060/ (Press CTRL+C to quit)
